In [12]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

import pandas as pd
import numpy as np
from sklearn.datasets import load_files

from datacode.retrieve_data import pull_training_data, pull_test_data
from nltk.corpus import stopwords
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
import spacy
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim

from features.pre_process import set_stop_words, sent_to_words, remove_stopwords, bigrams, get_corpus
import pickle
import warnings
from models.text_model import gen_lda_model, train_vectors



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/topper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Pull in training data, split off features and label data and save the features array as a df
imdb_train = pull_training_data("../data/raw/aclImdb/train")

text_train, y_train = imdb_train.data, imdb_train.target
text_train_df = pd.DataFrame({'text': text_train})

In [14]:
len(imdb_train)

5

# SAMPLING STEP - FOR TEST

In [22]:
text_train_df_samp = text_train_df.head(1000)

In [16]:
# Better to add to a YAML or other text file in due course
added_stop_words = ['film','movie','picture','review']

In [18]:
# Bringing in a pipeline pre-trained on written web text, includes vocab vectors, syntax and entities. 
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [28]:
# Run get_corpus which 
#train_corpus4, train_id2word4, bigram_train4 = get_corpus(text_train_df, added_stop_words)
train_corpus4, train_id2word4, bigram_train4 = get_corpus(text_train_df_samp, added_stop_words)  # REVERT AFTER TEST

In [40]:
# Keep the files
with open('../data/interim/train_corpus4.pkl', 'wb') as f:
    pickle.dump(train_corpus4, f)
with open('../data/interim/train_id2word4.pkl', 'wb') as f:
    pickle.dump(train_id2word4, f)
with open('../data/interim/bigram_train4.pkl', 'wb') as f:
    pickle.dump(bigram_train4, f)

In [2]:
with open('../data/interim/train_corpus4.pkl', 'rb') as f:
    train_corpus4 = pickle.load(f)
with open('../data/interim/train_id2word4.pkl', 'rb') as f:
    train_id2word4 = pickle.load(f)
with open('../data/interim/bigram_train4.pkl', 'rb') as f:
    bigram_train4 = pickle.load(f)

In [3]:
lda_train4 = gen_lda_model(train_corpus4, train_id2word4)

In [9]:
train_vecs = train_vectors(lda_train4, train_corpus4, text_train_df_samp )